# UDACITY Demo - Step Functions

In [1]:
import boto3
from sagemaker import get_execution_role

client = boto3.client('stepfunctions')

# Definitions are unweildly and also not valid python objects. Consider also using the Step interface
# for the services you wish to invoke, or just constructing a Step Function through the UI 
# and pasting the definition that is automatically generated. 

definition = """{
  "Comment": "A Hello World example of the Amazon States Language.",
  "StartAt": "Lambda Invoke (First)", 
  "States": {
    "Lambda Invoke (First)": {
      "Type": "Task",
      "Resource": "arn:aws:states:::lambda:invoke",
      "OutputPath": "$.Payload",
      "Parameters": {
        "Payload.$": "$",
        "FunctionName": "arn:aws:lambda:us-west-2:565094796913:function:example123:$LATEST"
      },
      "Retry": [
        {
          "ErrorEquals": [
            "Lambda.ServiceException",
            "Lambda.AWSLambdaException",
            "Lambda.SdkClientException"
          ],
          "IntervalSeconds": 2,
          "MaxAttempts": 6,
          "BackoffRate": 2
        }
      ],
      "Next": "Lambda Invoke (Second)"
    },
    "Lambda Invoke (Second)": {
      "Type": "Task",
      "Resource": "arn:aws:states:::lambda:invoke",
      "OutputPath": "$.Payload",
      "Parameters": {
        "Payload.$": "$",
        "FunctionName": "arn:aws:lambda:us-west-2:565094796913:function:example123:$LATEST"
      },
      "Retry": [
        {
          "ErrorEquals": [
            "Lambda.ServiceException",
            "Lambda.AWSLambdaException",
            "Lambda.SdkClientException"
          ],
          "IntervalSeconds": 2,
          "MaxAttempts": 6,
          "BackoffRate": 2
        }
      ],
      "End": true
    }
  }
}
"""

In [4]:
# Your SageMaker execution role needs to have both StepFunctions permission and IAM permission. This
# is seperate from the role passed in through 'roleArn' (You need IAM permissions specifically because
# you are handling other roles.) The roleArn that's passed into the state machine needs to have permissions
# to the services it's invoking. In this case, these are Lambda permissions. 

response = client.create_state_machine(
    name='lesson3-lecture4-boto3StateMachine', # Names need to be unique. 
    definition=definition,
    roleArn='arn:aws:iam::419056005463:role/service-role/StepFunctions-HelloWorld-role-94cbb165',
    type='STANDARD',
    loggingConfiguration={
        'level': 'OFF'
    }
)
print(response)

{'stateMachineArn': 'arn:aws:states:us-east-1:419056005463:stateMachine:lesson3-lecture4-boto3StateMachine', 'creationDate': datetime.datetime(2022, 5, 28, 12, 27, 19, 724000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '11470b28-f1df-4dfa-b457-6d57b51b10c0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '11470b28-f1df-4dfa-b457-6d57b51b10c0', 'date': 'Sat, 28 May 2022 10:27:19 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '139'}, 'RetryAttempts': 0}}


In [6]:
response = client.start_execution(
    stateMachineArn='arn:aws:states:us-east-1:419056005463:stateMachine:lesson3-lecture4-boto3StateMachine', # You can find this through the Console or through the 'response' object. 
    name='example-execution', # Execution names need to be unique within state machines. 
    input='{}' # Input needs to be at least empty brackets. 
)

In [7]:
print(response)

{'executionArn': 'arn:aws:states:us-east-1:419056005463:execution:lesson3-lecture4-boto3StateMachine:example-execution', 'startDate': datetime.datetime(2022, 5, 28, 12, 28, 18, 144000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '19666db8-67d7-494e-8b4a-14cb2e318914', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '19666db8-67d7-494e-8b4a-14cb2e318914', 'date': 'Sat, 28 May 2022 10:28:18 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '148'}, 'RetryAttempts': 0}}


**The execution of the state-machine is not successful, because the lambda-functions used need to be genereated in the personal environment.**